![Alt Text](axpo-animated.gif)
# Witamy na rozmowie rekrutacyjnej
## na stanowisko: **SME Analyst** w **Axpo Polska**

Cieszymy się, że zdecydowałeś/aś się wziąć udział w naszym procesie rekrutacyjnym. Przedstawiamy Ci case study, w którym zaprezentujesz swoje umiejętności analityczne.

---

## Opis Case Study

W bazie (tworzonej automatycznie przy uruchomieniu) znajdują się trzy tabele, przeznaczone do porównania danych między systemami:

- **hd_readings** – miesięczne odczyty zużyć dla ok. 500 POD-ów (medium_id: 1=power, 2=gas), zakres dat 2022-01 .. 2025-09
  - kolumna `loading_date` oznacza datę ładowania rekordów do tabeli. Dane dla całego portfela są appendowane do tabeli, całościowo, raz dziennie. 
  - `forecast_usage_mwh` oznacza prognozowane zuzycie danego POD w danym okresie 
  - `real_usage_mwh` oznacza rzeczywiste zuzycie danego POD w danym okresie
  - `product_id` oznacza produkt jaki był dostarczany danemu POD w danym okresie
  - `usage_from_date` - `usage_to_date` oznacza okres w jakim dostarczany był wolumen na danym POD
- **product_dict** – słownik produktów, nazwy zawierają kluczowe frazy: `gas_fix`, `gas_spot`, `power_fix`, `power_spot`, które będą identyfikowały typ produktu ENDUR 
- **endur_dump** – ENDUR jest systemem transakcyjnym. Tabela przedstawia agregat portfela na dane reporting date: (BOOK, endur_product, delivery_month, volume).
  - Agregat powstaje na podstawie wierszy z HD za danym loading_date, po poprawnym załadowaniu danych wolumeny między systemami powinny być zgodne.

---

## Zadania do Wykonania
- Uruchom komórkę poniżej – wygeneruje ona unikatowe dane i załaduje je do silnika SQL w pamięci (SQLite) oraz pozostawi jako DataFrame’y Pythona. 
- Sprawdź czy ładowanie danych do systemu Endur przebiegło dzisiaj poprawnie. Czy wolumeny rzeczywiste są zgodne na kazdym z produktów w kazdym z miesięcy? 
- Sprawdź, który endur_product ma najnizszy wspołczynnik ACFC (Actual/Forecast) - dlaczego jest on dla nas istotny i w jaki sposób mozemy poprawić jakość prognoz? 
- wylistuj POD-y, które zmieniały produkt w historii dostawy, wskaż miesiące zmiany i pomiedzy jakimi produktami była zmiana? 

## Dostępne narzędzia
- Analizę możesz przeprowadzić używając Pythona, SQL-a lub obu narzędzi.
- Możesz korzystać z internetu, ale nie używaj generatywnej AI.
- Bardziej niż wynik oceniany będzie sposób jego osiągnięcia oraz tok myślenia.

---
# Powodzenia!


In [31]:
import sqlite3
import pandas as pd
from IPython.display import display
try:
    from db_setup import setup_database, modify_random_usage
except ImportError as e:
    import importlib.util, os
    module_path = os.path.join(os.getcwd(), 'db_setup.py')
    if not os.path.exists(module_path):
        raise ImportError(f'Nie znaleziono db_setup.py w {os.getcwd()}') from e
    spec = importlib.util.spec_from_file_location('db_setup', module_path)
    db_setup = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(db_setup)
    setup_database = db_setup.setup_database
    modify_random_usage = getattr(db_setup, 'modify_random_usage', None)

hd_readings, product_dict, endur_dump = setup_database()

print("Przykładowe wiersze HD (tabela: hd_readings):")
display(hd_readings.head())

print("\nPrzykładowe wiersze słownik produktów (tabela: product_dict):")
display(product_dict.head())

print("\nPrzykładowe wiersze Endur (tabela: endur_dump):")
display(endur_dump.head())

if modify_random_usage is not None:
    info = modify_random_usage(hd_readings)

conn = sqlite3.connect(':memory:')
hd_readings.to_sql('hd_readings', conn, index=False)
product_dict.to_sql('product_dict', conn, index=False)
endur_dump.to_sql('endur_dump', conn, index=False)

def sqlquery(query, conn=conn):
    return pd.read_sql_query(query, conn)


Przykładowe wiersze HD (tabela: hd_readings):


,loading_date,pod_header_id,customer_id,adres_id,contrct_id,product_id,medium_id,usage_from_date,usage_to_date,forecast_usage_mwh,real_usage_mwh
0,2025-10-21,POD000413,CUST000157,ADDR000160,CTR000021,PRD026,2,2023-03-01,2023-03-31,31.677,39.414
1,2025-10-21,POD000194,CUST000315,ADDR000300,CTR000919,PRD051,1,2024-08-01,2024-08-31,7.496,6.145
2,2025-10-21,POD000300,CUST000342,ADDR000074,CTR001450,PRD089,1,2024-03-01,2024-03-31,20.630,19.731
3,2025-10-21,POD000427,CUST000047,ADDR000178,CTR001244,PRD082,1,2022-04-01,2022-04-30,0.637,0.570
4,2025-10-21,POD000158,CUST000111,ADDR000188,CTR000136,PRD069,1,2022-05-01,2022-05-31,28.156,32.537



Przykładowe wiersze słownik produktów (tabela: product_dict):


,product_id,product_name
0,PRD045,gas_spot_020_indexed
1,PRD071,smart_021_power_fix
2,PRD096,dynamic_021_power_spot
3,PRD029,gas_spot_004_indexed
4,PRD063,smart_013_power_fix



Przykładowe wiersze Endur (tabela: endur_dump):


,reporting_date,BOOK,endur_product,delivery_month,volume
0,2025-10-21,AXPL_SME_POWER,POWER_FIX,2024-04,3127.677
1,2025-10-21,AXPL_SME_POWER,POWER_SPOT,2024-10,3422.822
2,2025-10-21,AXPL_SME_POWER,POWER_SPOT,2022-08,3110.378
3,2025-10-21,AXPL_SME_GAS,GAS_FIX,2025-04,2314.427
4,2025-10-21,AXPL_SME_GAS,GAS_SPOT,2025-05,2245.769


In [35]:
# Przykładowy snippet SQL 
query = """
    SELECT 
        hr.* 
    FROM 
        hd_readings hr
    WHERE 
        1=1
        AND hr.medium_id = 1
    LIMIT 
        10
    """
sqlquery(query)
        

,loading_date,pod_header_id,customer_id,adres_id,contrct_id,product_id,medium_id,usage_from_date,usage_to_date,forecast_usage_mwh,real_usage_mwh
0,2025-10-21,POD000194,CUST000315,ADDR000300,CTR000919,PRD051,1,2024-08-01,2024-08-31,7.496,6.145
1,2025-10-21,POD000300,CUST000342,ADDR000074,CTR001450,PRD089,1,2024-03-01,2024-03-31,20.630,19.731
2,2025-10-21,POD000427,CUST000047,ADDR000178,CTR001244,PRD082,1,2022-04-01,2022-04-30,0.637,0.570
3,2025-10-21,POD000158,CUST000111,ADDR000188,CTR000136,PRD069,1,2022-05-01,2022-05-31,28.156,32.537
4,2025-10-21,POD000313,CUST000193,ADDR000233,CTR001033,PRD089,1,2022-11-01,2022-11-30,24.177,21.275
5,2025-10-21,POD000062,CUST000128,ADDR000236,CTR001099,PRD055,1,2025-01-01,2025-01-31,6.642,5.584
6,2025-10-21,POD000190,CUST000091,ADDR000274,CTR001078,PRD059,1,2024-07-01,2024-07-31,7.171,8.856
7,2025-10-21,POD000143,CUST000104,ADDR000230,CTR000106,PRD063,1,2023-12-01,2023-12-31,7.716,5.992
8,2025-10-21,POD000205,CUST000250,ADDR000105,CTR000784,PRD064,1,2025-06-01,2025-06-30,34.311,33.053
9,2025-10-21,POD000252,CUST000264,ADDR000194,CTR000300,PRD069,1,2024-01-01,2024-01-31,33.414,42.052
